# access thredds manual

getting different regions of current data manually instead of using the auto-generated regions.

In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import xarray as xr

from utils import xr_dataset_to_fieldset

In [ ]:
dataset_url_6kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd"
dataset_url_2kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd"
dataset_url_1kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/1km/hourly/RTV/HFRADAR_US_West_Coast_1km_Resolution_Hourly_RTV_best.ncd"

DATA_6KM = 6
DATA_2KM = 2
DATA_1KM = 1

filename_dict = {
    DATA_6KM: "west_coast_6km_hourly",
    DATA_2KM: "west_coast_2km_hourly",
    DATA_1KM: "west_coast_1km_hourly"
}

def get_region(data):
    time_range = data[2]
    if isinstance(time_range[0], str):
        time_range = (np.datetime64(time_range[0]), np.datetime64(time_range[1]))
    lat_range = data[3]
    lon_range = data[4]
    return dict(
        name = data[0],
        category = data[1],
        time = time_range,
        lat = lat_range,
        lon = lon_range,
        domain = {
            "S": lat_range[0],
            "N": lat_range[1],
            "W": lon_range[0],
            "E": lon_range[1],
        }
    )

In [ ]:
num_chunks = 50

thredds_data = {
    DATA_6KM: xr.open_dataset(dataset_url_6kmhourly, chunks={"time": num_chunks}),
    DATA_2KM: xr.open_dataset(dataset_url_2kmhourly, chunks={"time": num_chunks}),
    DATA_1KM: xr.open_dataset(dataset_url_1kmhourly, chunks={"time": num_chunks})
}

In [ ]:
region_data = [
    ("tijuana_river", DATA_2KM, ("2020-06-16T21", "2020-06-23T21"), (32.528, 32.71), (-117.29, -117.11)),
    ("tijuana_river", DATA_1KM, ("2020-06-16T21", "2020-06-23T21"), (32.528, 32.71), (-117.29, -117.11))
]

In [ ]:
regions = []
for rd in region_data:
    new_reg = get_region(rd)
    new_reg["dataset"] = thredds_data[new_reg["category"]].sel(
        time=slice(new_reg["time"][0], new_reg["time"][1]),
        lat=slice(new_reg["lat"][0], new_reg["lat"][1]),
        lon=slice(new_reg["lon"][0], new_reg["lon"][1]),
    )
    regions.append(new_reg)
    print(f"region {new_reg['name']} data megabytes: {new_reg['dataset'].nbytes / 1024 / 1024}")

In [ ]:
for r in regions:
    save_dir = filename_dict[r["category"]] + "_netcdfs/"
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        print(f"created new directory {save_dir}")
    filename = filename_dict[r['category']] + f"_region_{r['name']}.nc"
    # save file
    r["dataset"].to_netcdf(save_dir + filename)
    print(f"saved to {save_dir + filename}")
print("done")

## fieldset testing

In [ ]:
from parcels import FieldSet

In [ ]:
netcdf_filename = "west_coast_1km_hourly_netcdfs/west_coast_1km_hourly_region_tijuana_river.nc"
ds = xr.open_dataset(netcdf_filename)
fs = xr_dataset_to_fieldset(ds)
ds.close()
fs.U.show()